Unzip

In [ ]:
!unzip Localization_project-main.zip
%ls

Archive:  Localization_project-main.zip
   creating: Localization_project-main/
   creating: Localization_project-main/Ground/
   creating: Localization_project-main/Ground/all_gt/
  inflating: Localization_project-main/Ground/all_gt/1.csv  
  inflating: Localization_project-main/Ground/all_gt/10.csv  
  inflating: Localization_project-main/Ground/all_gt/11.csv  
  inflating: Localization_project-main/Ground/all_gt/12.csv  
  inflating: Localization_project-main/Ground/all_gt/13.csv  
  inflating: Localization_project-main/Ground/all_gt/14.csv  
  inflating: Localization_project-main/Ground/all_gt/15.csv  
  inflating: Localization_project-main/Ground/all_gt/16.csv  
  inflating: Localization_project-main/Ground/all_gt/17.csv  
  inflating: Localization_project-main/Ground/all_gt/18.csv  
  inflating: Localization_project-main/Ground/all_gt/19.csv  
  inflating: Localization_project-main/Ground/all_gt/2.csv  
  inflating: Localization_project-main/Ground/all_gt/20.csv  
  inflating: Lo

RSSI

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, Kernel, Hyperparameter, RBF, ConstantKernel, RationalQuadratic, Matern, ExpSineSquared, DotProduct
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean, cdist
import scipy

# Utility to split data into train and test sets
def testAndTrainData(df1, df2):
    tmp1 = df1['location'].to_numpy()
    y1 = df1['rssi_mean']
    y_train = y1.to_numpy()
    X1 = [list(map(int, i.split(","))) for i in tmp1]
    X_train = np.array(X1)

    tmp2 = df2['location'].to_numpy()
    y2 = df2['rssi_mean']
    y_test = y2.to_numpy()
    X2 = [list(map(int, i.split(","))) for i in tmp2]
    X_test = np.array(X2)

    return X_train, y_train, X_test, y_test


# Epanechnikov kernel for GPR
class EpanechnikovKernel(Kernel):
    def __init__(self, length_scale=1.0, length_scale_bounds=(1e-5, 1e5)):
        self.length_scale = length_scale
        self.length_scale_bounds = length_scale_bounds

    @property
    def hyperparameter_length_scale(self):
        return Hyperparameter("length_scale", "numeric", self.length_scale_bounds)

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        length_scale = self.length_scale

        if Y is None:
            Y = X
        else:
            Y = np.atleast_2d(Y)

        # Pairwise Euclidean distances
        dists_sq = np.sum(X**2, axis=1)[:, np.newaxis] + \
                   np.sum(Y**2, axis=1)[np.newaxis, :] - \
                   2 * np.dot(X, Y.T)
        dists = np.sqrt(np.maximum(dists_sq, 1e-12))
        dists_normalized = dists / length_scale

        # Epanechnikov kernel
        K = (3 / 4) * np.maximum(1 - dists_normalized**2, 0)

        if eval_gradient:
            mask = (dists_normalized <= 1)
            dK_dl = (3 / 4) * (2 * dists_normalized**2 / length_scale) * mask
            return K, dK_dl[..., np.newaxis]
        else:
            return K

    def diag(self, X):
        return np.full(X.shape[0], 0.75)

    def is_stationary(self):
        return True

    def __repr__(self):
        return f"EpanechnikovKernel(length_scale={self.length_scale})"

def calculate_distances(X):
    """Calculate distances from origin and center for enhanced features"""
    center = np.array([121.5, 130])  # Center of the grid (242x259)
    distances = np.array([
        [euclidean(point, [0, 0]), euclidean(point, center)]
        for point in X
    ])
    return distances



# Main processing loop
floor = '/content/Localization_project-main/Ground'
folder_path = floor + '/beacons_gt/'

test_location = [
    "152, 92", "73, 164", "103, 80", "105, 51", "166, 106", "224, 124",
    "43, 209", "184, 126", "238, 135", "113, 80", "70, 191", "140, 217",
    "81, 18", "186, 250", "156, 134", "202, 181", "64, 164", "177, 159",
    "116, 53", "105, 53", "116, 60", "152, 80", "132, 60", "132, 51",
    "132, 70", "132, 80", "152, 51", "140, 80", "178, 96", "156, 118",
    "178, 118", "116, 80", "152, 90", "147, 87", "150, 80"
]

seasonal_kernel = (2.0 ** 2 * RBF(length_scale=100.0) * ExpSineSquared(length_scale=1.0, periodicity=1.0, periodicity_bounds="fixed"))
irregularities_kernel = 0.5 ** 2 * RationalQuadratic(length_scale=1.0, alpha=1.0)

# Kernel candidates to test
kernel_candidates = [
     Matern(nu = 0.1) + EpanechnikovKernel(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    Matern(nu = 1.1) + EpanechnikovKernel(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    Matern(nu = 2.0) + EpanechnikovKernel(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
DotProduct() + Matern(nu=0.1) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    DotProduct() + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
  #   Matern(nu = 2.5) + EpanechnikovKernel(length_scale=1.0)
  #   + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5))
  #   + seasonal_kernel + irregularities_kernel,
  #   Matern(nu = 1.5) + EpanechnikovKernel(length_scale=1.0)
  #   + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5))
  #   + seasonal_kernel + irregularities_kernel,
  # DotProduct() + RBF(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5))
  #   + seasonal_kernel + irregularities_kernel,
  #   DotProduct() + EpanechnikovKernel(length_scale=1.0)
  #   + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5))
  #   + seasonal_kernel+ irregularities_kernel,
  #   DotProduct() + Matern(nu=1.5) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5))
  #   + seasonal_kernel + irregularities_kernel,
  #   DotProduct() + Matern(nu=2.5) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5))
  #   + seasonal_kernel + irregularities_kernel
]

for filename in os.listdir(folder_path):
    print(f"Processing {filename}")
    if not filename.endswith('.csv'):
        continue

    # Load data
    df = pd.read_csv(os.path.join(folder_path, filename))
    v = filename.split('.csv')[0]

    # Split data
    df1 = df.loc[~df['location'].isin(test_location)]
    df2 = df.loc[df['location'].isin(test_location)].sort_values(by=['location'])
    X_train, y_train, X_test, y_test = testAndTrainData(df1, df2)

    # Feature engineering: Add distance-based features
    X_train_dist = calculate_distances(X_train)
    X_test_dist = calculate_distances(X_test)

    # Combine coordinates with distance features
    X_train_extended = np.hstack((X_train, X_train_dist))
    X_test_extended = np.hstack((X_test, X_test_dist))

    # Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_extended)
    X_test_scaled = scaler.transform(X_test_extended)

    # Track best model for this beacon
    best_mae = float('inf')
    best_gp = None
    best_kernel = None

    # Try each kernel candidate
    for kernel in kernel_candidates:
        gp = GaussianProcessRegressor(
            kernel=kernel,
            n_restarts_optimizer=15,
            normalize_y=True,
            alpha=0.01  # Small noise for stability
        )

        # Fit and predict
        gp.fit(X_train_scaled, y_train)
        y_pred, _ = gp.predict(X_test_scaled, return_std=True)

        # Calculate MAE
        current_mae = mean_absolute_error(y_test, y_pred)

        # Track best kernel
        if current_mae < best_mae:
            best_mae = current_mae
            best_gp = gp
            best_kernel = kernel

    print(f"Best kernel for {v}: {best_kernel} with MAE: {best_mae:.2f}")

    # ================== DYNAMIC MESHGRID GENERATION ==================
    # Combine all data points to determine bounds
    all_points = np.vstack((X_train, X_test))

    # Get min/max coordinates with buffer
    min_x, min_y = np.min(all_points, axis=0)
    max_x, max_y = np.max(all_points, axis=0)

    # Add 10% buffer to the bounding box
    x_buffer = max(1, int(0.1 * (max_x - min_x)))
    y_buffer = max(1, int(0.1 * (max_y - min_y)))

    # Define bounds with buffer (clamped to 1-242/259 range)
    min_x = max(1, min_x - x_buffer)
    max_x = min(242, max_x + x_buffer)
    min_y = max(1, min_y - y_buffer)
    max_y = min(259, max_y + y_buffer)

    # Create dynamic meshgrid within bounds
    x1 = np.arange(min_x, max_x + 1).astype(int)
    x2 = np.arange(min_y, max_y + 1).astype(int)
    X0p, X1p = np.meshgrid(x1, x2)
    grid_points = np.c_[X0p.ravel(), X1p.ravel()]

    # Add distance features to grid points
    grid_distances = calculate_distances(grid_points)
    grid_points_extended = np.hstack((grid_points, grid_distances))
    grid_points_scaled = scaler.transform(grid_points_extended)

    # Predict using best model
    mean_pred, std_pred = best_gp.predict(grid_points_scaled, return_std=True)

    # Create full grid arrays with -99 default
    full_x = np.arange(1, 243)  # 1-242
    full_y = np.arange(1, 260)  # 1-259
    full_X0p, full_X1p = np.meshgrid(full_x, full_y)
    full_grid_points = np.c_[full_X0p.ravel(), full_X1p.ravel()]

    # Initialize full RSSI arrays with -99
    full_mean_pred = np.full(len(full_grid_points), -102.0)
    full_std_pred = np.full(len(full_grid_points), -102.0)

    # Create mapping from coordinate to index
    coord_to_index = {(x, y): i for i, (x, y) in enumerate(full_grid_points)}

    # Map predictions to full grid
    for (x, y), mean_val, std_val in zip(grid_points, mean_pred, std_pred):
        idx = coord_to_index.get((x, y), None)
        if idx is not None:
            full_mean_pred[idx] = mean_val
            full_std_pred[idx] = std_val

    # Prepare data for saving
    data = [
        [f"{x}, {y}", round(mean, 2), round(std, 2)]
        for (x, y), mean, std in zip(full_grid_points, full_mean_pred, full_std_pred)
    ]
    # ================== END DYNAMIC MESHGRID ==================

    df_out = pd.DataFrame(data, columns=["location", "rssi_mean", "rssi_std"])
    df_out.to_csv(f'/content/Localization_project-main/Ground/beacons_pd/{v}.csv', index=False)

    # Final evaluation
    y_pred, _ = best_gp.predict(X_test_scaled, return_std=True)
    final_mae = mean_absolute_error(y_test, y_pred)
    print(f"  Final MAE: {final_mae:.2f}")
    print(f"  Grid size: Original (242x259) | Dynamic: ({min_x}-{max_x})x({min_y}-{max_y})")

Processing IW25030290.csv


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k2__k1__constant_value is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11

Best kernel for IW25030290: Matern(length_scale=1, nu=0.1) + EpanechnikovKernel(length_scale=1.0) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 1.39
  Final MAE: 1.39
  Grid size: Original (242x259) | Dynamic: (59-79)x(70-191)
Processing IW25030183.csv


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k2__k1__constant_value is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k2__k1__constant_value is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/

Best kernel for IW25030183: Matern(length_scale=1, nu=1.1) + EpanechnikovKernel(length_scale=1.0) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 3.43
  Final MAE: 3.43
  Grid size: Original (242x259) | Dynamic: (23-161)x(115-231)
Processing IW25030321.csv


...

Best kernel for IW25030302: Matern(length_scale=1, nu=1.1) + EpanechnikovKernel(length_scale=1.0) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 5.52
  Final MAE: 5.52
  Grid size: Original (242x259) | Dynamic: (175-204)x(139-195)
Processing IW25030313.csv


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k1__k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k1__k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-package

Best kernel for IW25030313: Matern(length_scale=1, nu=2) + EpanechnikovKernel(length_scale=1.0) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 4.76
  Final MAE: 4.76
  Grid size: Original (242x259) | Dynamic: (96-185)x(41-125)
Processing IW25030307.csv


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__k1__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11

Best kernel for IW25030307: Matern(length_scale=1, nu=2) + EpanechnikovKernel(length_scale=1.0) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 4.19
  Final MAE: 4.19
  Grid size: Original (242x259) | Dynamic: (117-227)x(41-169)


Localization_2

In [ ]:
#!/usr/bin/env python
import os
from IPython.display import display
from itertools import product
import numpy as np
from sklearn.metrics import mean_absolute_error
import pandas as pd
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import accuracy_score
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C,WhiteKernel , RationalQuadratic ,ExpSineSquared
# %matplotlib notebook


# Directory containing beacon files
def GroundTruthData(directory_gt, floor,test_location,num_files):

    # Initialize an empty list to hold dataframes
    dataframes_gt = []
    # Loop through each beacon file
    for filename in os.listdir(directory_gt):
        # print(filename)
        file_path = os.path.join(directory_gt, filename)
        df = pd.read_csv(file_path)
        df.rename(columns={'rssi_mean':  filename.split('.csv')[0]}, inplace=True)
        #df = df.drop(['rssi_std'], axis=1) # Drop 'rssi_std' column
        dataframes_gt.append(df)  # Append dataframe to the list

    dataframe_gt = pd.concat([df.set_index('location') for df in dataframes_gt], axis=1, join='outer').reset_index()
    dataframe_gt.fillna('-97.7', inplace=True)
    # print(dataframe_gt.round(0))
    dataframe_gt.to_csv(floor+'/all_scan_sample.csv', index=False)

    ### Randomly sampled 6 test location out of total 29 location and 3 outlier = 32 points####
    test_data = dataframe_gt.loc[dataframe_gt['location'].isin(test_location)]
    test_data = test_data.sort_values(by=['location'])
    temp = test_data.location.str.split(",", n = -1, expand = True)
    # print(dataframe_gt)
    test_data['cordX'] = temp[0].astype(int)
    test_data['cordY'] = temp[1].astype(int)
    # print(data.head())
    X_gt= test_data.iloc[:, 1:num_files]
    y1_gt = test_data['cordX']
    y2_gt = test_data['cordY']
    test_data.head()
    test_data.shape

    return X_gt, y1_gt, y2_gt


def Custom_PD_data(directory_pd, floor,test_location,num_files):

    # Initialize an empty list to hold dataframes
    dataframes_pd = []
    # Loop through each beacon file
    for filename in os.listdir(directory_pd):
        # print(filename)
        file_path = os.path.join(directory_pd, filename)
        df = pd.read_csv(file_path)
        df.rename(columns={'rssi_mean':  filename.split('.csv')[0]}, inplace=True)
        df = df.drop(['rssi_std'], axis=1) # Drop 'rssi_std' column
        dataframes_pd.append(df)  # Append dataframe to the list

    dataframe_pd = pd.concat([df.set_index('location') for df in dataframes_pd], axis=1, join='inner').reset_index()

    ### Randomly sampled 6 test location out of total 29 location and 3 outlier = 32 points####
    test_data = dataframe_pd.loc[dataframe_pd['location'].isin(test_location)]
    test_data = test_data.sort_values(by=['location'])
    temp = test_data.location.str.split(",", n = -1, expand = True)
    test_data['cordX'] = temp[0].astype(int)
    test_data['cordY'] = temp[1].astype(int)
    # print(data.head())
    X_pd_custom= test_data.iloc[:, 1:num_files]
    y1_pd_custom = test_data['cordX']
    y2_pd_custom = test_data['cordY']
    test_data.head()
    test_data.shape

    return X_pd_custom, y1_pd_custom, y2_pd_custom

### beacons_pd folder include predicted beacons rssi data ########
def GPPredictedData(directory_pd, num_files):
    dataframes_pd = []
    for filename in os.listdir(directory_pd):
        file_path = os.path.join(directory_pd, filename)
        # Read CSV file
        df = pd.read_csv(file_path)
        # Rename column
        df.rename(columns={'rssi_mean':  filename.split('.csv')[0]}, inplace=True)
        # Drop 'rssi_std' column
        df = df.drop(['rssi_std'], axis=1)
        # Append dataframe to the list
        dataframes_pd.append(df)
    # Merge all dataframes on 'location'
    dataframes_pd = pd.concat([df.set_index('location') for df in dataframes_pd], axis=1, join='inner').reset_index()
    new = dataframes_pd.location.str.split(",", n = -1, expand = True)
    dataframes_pd['cordX'] = new[0].astype(int)
    dataframes_pd['cordY'] = new[1].astype(int)
    X_pd = dataframes_pd.iloc[:, 1:num_files]
    y1_pd = dataframes_pd['cordX']
    y2_pd = dataframes_pd['cordY']

    dataframes_pd.head()
    dataframes_pd.shape

    return X_pd, y1_pd, y2_pd


def ContinuousData(directory_pd, num_files):
    dataframes_pd = []

        # Read CSV file
    dataframes_pd = pd.read_csv(file_path)
        # Rename column

    # Merge all dataframes on 'location'
    #dataframes_pd = pd.concat([df.set_index('location') for df in dataframes_pd], axis=1, join='inner').reset_index()
    new = dataframes_pd.location.str.split(",", n = -1, expand = True)
    dataframes_pd['cordX'] = new[0].astype(int)
    dataframes_pd['cordY'] = new[1].astype(int)
    X_cd = dataframes_pd.iloc[:, 1:num_files]
    y1_cd = dataframes_pd['cordX']
    y2_cd = dataframes_pd['cordY']

    dataframes_pd.head()
    dataframes_pd.shape

    return X_cd, y1_cd, y2_cd

#################### KNN training for X-axis ####################
def knnXaxis(X_train, y1_train, X_test,y1_test, y1_gt):
    print("---------KNN training for X-axis result---------" )
    # print(X_test, y1_test)
    knn_classifier = KNeighborsClassifier(n_neighbors = 3)
    # print(X_train, y1_train)
    knn_classifier.fit(X_train, y1_train.values.ravel())
    # test using GP predicted test dataset
    #y1_pred=knn_classifier.predict(X_test)
    y1_pred_custom=knn_classifier.predict(X_pd_custom)
    # print(y1_pred_custom)
    # print(y1_pd_custom)
    # knn_accuracy=accuracy_score(y1_test, y1_pred)
    # print('\nThe Models Accuracy is', knn_accuracy)
    MAE1 = mean_absolute_error(y1_pred,y1_test)
    MAE2 = mean_absolute_error(y1_pred_custom,y1_pd_custom)
    print("Predicted data knn-mae-X-axis :\t",MAE1*0.305, "m")
    print("Custom Predicted data knn-mae-X-axis (0 to 15 ft):\t",MAE2*0.305, "m")
    # test using ground truth test dataset
    # print("Inputs==========",X_gt)
    y1_pred_gt=knn_classifier.predict(X_gt)
    # print("Outputs======",y1_pred_gt)
    # # knn_accuracy=accuracy_score(y1_gt, y1_pred_gt)
    # # print('\nThe Models Accuracy for ground truth values, is', knn_accuracy)
    # print(y1_gt,y1_pred_gt)
    MAE3 = mean_absolute_error(y1_pred_gt,y1_gt)
    #print("Ground truth data knn-mae-X-axis :\t",MAE3*0.305, "m")

    return MAE1, MAE2, MAE3, y1_pred, y1_pred_custom, y1_pred_gt


#################### KNN training for Y-axis ####################
def knnYaxis(X_train, y2_train, X_test, y2_test,y2_gt):
    print("---------KNN training for Y-axis result---------" )
    knn_classifier = KNeighborsClassifier(n_neighbors = 3)
    knn_classifier.fit(X_train, y2_train.values.ravel())
    # test using GP predicted test dataset
    y2_pred=knn_classifier.predict(X_test)
    y2_pred_custom=knn_classifier.predict(X_pd_custom)
    #print(y2_pred)
    # knn_accuracy=accuracy_score(y2_test, y2_pred)
    # print('\nThe Models Accuracy is', knn_accuracy)

    MAE4 = mean_absolute_error(y2_pred,y2_test)
    MAE5 = mean_absolute_error(y2_pred_custom,y2_pd_custom)
    print("Predicted data knn-mae-Y-axis :\t",MAE4*0.305, "m")
    print("Custom Predicted data knn-mae-Y-axis (0 to 15 ft):\t",MAE5*0.305, "m")
    # test using ground truth test dataset
    y2_pred_gt=knn_classifier.predict(X_gt)
    MAE6 = mean_absolute_error(y2_pred_gt,y2_gt)
    # print(y2_gt,y2_pred_gt)
    #print("Ground truth data knn-mae-Y-axis :\t",MAE6*0.305, "m")

    return MAE4, MAE5, MAE6, y2_pred, y2_pred_custom, y2_pred_gt


########## KNN - Localization performance #########################
########### Part 1: Result on GP predicted test dataset ##################
def knnLocalizationPredictedData(y1_test, y2_test, X_test, y1_pred, y2_pred, path_3):
    r1 = pd.concat([y1_test, y2_test, X_test], axis=1)
    # r1.rename(columns={'cordX': 'cordXold', 1: 'cordYold'}, inplace=True)
    # r1.drop(columns=["cordX", "cordY"], inplace=True)
    r2 = pd.concat([pd.DataFrame({'SP_cordX':y1_pred}),pd.DataFrame({'SP_cordY':y2_pred})], axis=1)
    result = pd.concat([r1,r2.set_index(r1.index)], axis=1)
    #display(result)
    # euclidean distance error in a grid world
    # result = result.rename(columns={0: 'cordXold', 1: 'cordYold'}, inplace=True)
    result['dist_localization_error_pd']  = np.round(np.sqrt((result['cordX'] - result['SP_cordX'])**2 + (result['cordY'] - result['SP_cordY'])**2),2)
    result['loc'] = (result['cordX'].map(str)).str.cat(result['cordY'].map(str),sep=",")
    result['Pred_loc'] = (result['SP_cordX'].map(str)).str.cat(result['SP_cordY'].map(str),sep=",")
    result = result[['loc','Pred_loc','dist_localization_error_pd']]
    # print("Part 1: Result on GP predicted test dataset ==========")

    print("Localziation_mse_knn_pd_predicted", np.mean(result['dist_localization_error_pd'])*0.305, "m")
    # print("Localization error WRT Predicted data", result['dist_localization_error_pd']*0.305, "m")
    #result.to_csv(path_3+'/knn_result_pd_predicted.csv', index=False)

    return np.mean(result['dist_localization_error_pd'])


def knnLocalizationcustomPD(y1_pd_custom, y2_pd_custom, X_pd_custom, y1_pred_custom, y2_pred_custom):
    r1 = pd.concat([y1_pd_custom, y2_pd_custom, X_pd_custom], axis=1)
    # r1.rename(columns={'cordX': 'cordXold', 1: 'cordYold'}, inplace=True)
    # r1.drop(columns=["cordX", "cordY"], inplace=True)
    r2 = pd.concat([pd.DataFrame({'SP_cordX':y1_pred_custom}),pd.DataFrame({'SP_cordY':y2_pred_custom})], axis=1)
    result = pd.concat([r1,r2.set_index(r1.index)], axis=1)
    #display(result)
    # euclidean distance error in a grid world
    # result = result.rename(columns={0: 'cordXold', 1: 'cordYold'}, inplace=True)
    result['dist_localization_error_pd']  = np.round(np.sqrt((result['cordX'] - result['SP_cordX'])**2 + (result['cordY'] - result['SP_cordY'])**2),2)
    result['loc'] = (result['cordX'].map(str)).str.cat(result['cordY'].map(str),sep=",")
    result['Pred_loc'] = (result['SP_cordX'].map(str)).str.cat(result['SP_cordY'].map(str),sep=",")
    result = result[['loc','Pred_loc','dist_localization_error_pd']]
    # print("Part 1: Result on GP predicted test dataset ==========")

    print("Localziation_mse_knn_custom_pd_predicted (0 to 15 ft)", np.mean(result['dist_localization_error_pd'])*0.305, "m")
    # print("Localization error WRT Predicted data", result['dist_localization_error_pd']*0.305, "m")
    #result.to_csv(path_3+'/knn_result_pd_predicted.csv', index=False)

    return np.mean(result['dist_localization_error_pd'])



########## Part 2: Result on Ground truth test dataset ###################
def knnLocalizationGroundTruthData(y1_gt, y2_gt, X_gt, y1_pred_gt, y2_pred_gt, path_4):
    r1 = pd.concat([y1_gt, y2_gt, X_gt], axis=1)
    r2 = pd.concat([pd.DataFrame({'GPP_cordX':y1_pred_gt}),pd.DataFrame({'GPP_cordY':y2_pred_gt})], axis=1)
    result = pd.concat([r1,r2.set_index(r1.index)], axis=1)
    #print(result)
    # euclidean distance error in a grid world
    result['dist_localization_error_gt']  = np.round(np.sqrt((result['cordX'] - result['GPP_cordX'])**2 + (result['cordY'] - result['GPP_cordY'])**2),2)
    result['loc'] = (result['cordX'].map(str)).str.cat(result['cordY'].map(str),sep=",")
    result['Pred_loc'] = (result['GPP_cordX'].map(str)).str.cat(result['GPP_cordY'].map(str),sep=",")
    result = result[['loc','Pred_loc','dist_localization_error_gt']]
    # print("Part 2: Result on Ground truth test dataset ==========")

    print("Localziation_mse_knn_gt_predicted", np.mean(result['dist_localization_error_gt'])*0.305, "m")
    # print("Localization error WRT Ground truth data", result['dist_localization_error_gt']*0.305, "m")
    #result.to_csv(path_4+'/knn_result_gt_predicted.csv', index=False)

    return np.mean(result['dist_localization_error_gt'])


floor = '/content/Localization_project-main/Ground'
path_1 ='/content/Localization_project-main/Ground/beacons_gt'

test_location_pd = [
    "167, 242", "197, 185", "78, 70", "133, 63", "153, 111", "170, 230", "63, 147", "205, 244", "79, 76", "157, 117", "152, 121", "185, 200", "42, 219", "92, 217", "165, 107", "173, 153", "61, 100", "194, 114", "155, 89", "120, 77", "129, 49", "189, 166", "48, 217", "244, 228", "69, 179", "109, 216", "172, 113", "151, 120", "87, 79", "157, 80", "189, 170", "38, 222", "154, 134", "76, 152", "190, 238", "244, 233", "84, 156", "56, 98", "69, 112", "164, 249", "87, 218", "47, 83", "82, 60", "157, 191", "174, 222", "189, 119", "205, 175", "69, 158", "203, 221", "116, 49", "177, 136", "174, 215", "249, 138", "150, 66", "159, 194", "172, 114", "225, 120", "123, 220", "68, 115", "91, 78", "39, 220", "79, 32", "15, 79", "54, 78", "180, 199", "56, 79", "195, 212", "195, 239", "210, 117", "133, 66", "181, 242", "83, 63", "196, 188", "220, 210", "236, 144", "84, 151", "88, 158", "84, 18", "175, 214", "137, 216", "234, 125", "72, 151", "82, 59", "154, 84", "125, 217", "222, 205", "202, 218", "203, 175", "107, 216", "169, 258", "191, 215", "47, 80", "148, 49", "167, 228", "46, 218", "165, 93", "240, 125", "172, 153", "225, 216", "237, 148",
    "69, 224", "190, 233", "86, 226", "226, 118", "249, 147", "202, 191", "182, 181", "197, 110", "124, 63", "145, 214", "86, 35", "90, 212", "2, 90", "59, 175", "211, 180", "189, 87", "38, 192", "165, 130", "74, 207", "240, 116", "74, 116", "147, 49", "106, 38", "161, 83", "27, 88", "163, 195", "202, 212", "155, 213", "78, 24", "126, 50", "190, 160", "76, 206", "76, 53", "56, 226", "69, 222", "175, 252", "144, 121", "145, 50", "190, 196", "123, 57", "190, 180", "77, 23", "93, 149", "216, 220", "66, 224", "89, 60", "55, 120", "86, 37", "165, 195", "123, 49", "96, 224", "73, 47", "154, 183", "244, 208", "17, 71", "147, 73", "231, 150", "48, 191", "99, 68", "188, 182", "193, 168", "217, 128", "205, 240", "76, 29", "174, 228", "185, 112", "182, 148", "76, 72", "127, 43", "194, 166", "137, 74", "73, 205", "62, 180", "148, 188", "164, 126", "187, 188", "51, 74", "233, 219", "190, 185", "122, 56", "120, 86", "23, 84", "155, 139", "98, 81", "74, 113", "112, 88", "190, 203", "59, 142", "147, 136", "211, 127", "93, 224", "128, 42", "200, 192", "60, 176", "71, 219", "33, 208", "70, 114", "85, 44", "183, 154", "70, 224",
    "99, 157", "177, 82", "139, 130", "81, 92", "178, 181", "113, 34", "139, 127", "92, 43", "54, 170", "100, 154", "53, 90", "63, 228", "207, 190", "147, 40", "118, 228", "245, 205", "123, 227", "206, 205", "100, 156", "54, 198", "52, 153", "211, 190", "158, 181", "77, 207", "230, 233", "144, 113", "160, 207", "196, 108", "3, 70", "143, 134", "33, 229", "84, 229", "241, 114", "157, 206", "238, 240", "243, 115", "144, 43", "176, 180", "144, 106", "80, 109", "179, 175", "165, 147", "175, 83", "146, 194", "141, 117", "220, 196", "226, 196", "234, 112", "103, 206", "111, 205", "114, 34", "168, 196", "152, 39", "50, 103", "168, 186", "240, 155", "71, 67", "164, 67", "139, 228", "31, 206", "187, 230", "69, 27", "139, 96", "168, 85", "61, 229", "121, 226", "173, 84", "31, 207", "215, 176", "206, 192", "23, 223", "54, 109", "76, 140", "175, 173", "212, 203", "76, 118", "166, 141", "200, 164", "60, 193", "22, 220", "82, 90", "208, 165", "151, 144", "71, 54", "28, 207", "98, 146", "167, 128", "75, 120", "187, 151", "56, 229", "54, 126", "26, 92", "49, 101", "221, 135", "204, 227", "214, 206", "172, 187", "81, 102", "144, 138", "21, 69"
]
test_location_ground = ["152, 92", "116, 53", "105, 53", "116, 60", "152, 80", "132, 60",
    "132, 51", "132, 70", "132, 80", "152, 51", "140, 80", "166, 106",
    "178, 96", "156, 118", "178, 118", "116, 80", "152, 90", "147, 87",
    "150, 80", "152, 70", "152, 60", "142, 57", "132, 57", "126, 80",
    "113, 80", "116, 70", "110, 53", "105, 51", "176, 118", "174, 114",
    "178, 106", "166, 118", "163, 111", "156, 108", "156, 98", "168, 96",
    "174, 100", "163, 103", "167, 107", "183, 91", "150, 124", "177, 159",
    "177, 133", "185, 125", "184, 126", "192, 118", "218, 118", "202, 118",
    "212, 118", "177, 143", "177, 153", "93, 80", "103, 80", "91, 80",
    "43, 80", "110, 48", "73, 164", "64, 106", "70, 108", "88, 154",
    "70, 171", "64, 164", "64, 154", "64, 171", "64, 133", "90, 164",
    "60, 100", "70, 184", "64, 100", "64, 118", "64, 128", "64, 143",
    "64, 153", "68, 154", "78, 154", "64, 160", "70, 181", "80, 164",
    "80, 174", "81, 80", "64, 80", "74, 80", "64, 93", "18, 80", "93, 69",
    "54, 80", "47, 80", "33, 80", "81, 50", "81, 60", "81, 70", "81, 30",
    "28, 80", "156, 134", "156, 191", "182, 191", "195, 191", "70, 191",
    "70, 199", "43, 199", "43, 212", "43, 209", "66, 217", "96, 217",
    "9, 80", "9, 90", "94, 53", "110, 44", "95, 53", "81, 40", "231, 131",
    "238, 124", "238, 135", "238, 145", "224, 124", "195, 199", "182, 199",
    "195, 181", "202, 181", "170, 217", "168, 246", "168, 253", "203, 246",
    "182, 217", "195, 217", "203, 217", "224, 208", "242, 229", "242, 217",
    "224, 217", "184, 166", "191, 173", "202, 176", "172, 217", "170, 224",
    "170, 234", "164, 253", "166, 257", "202, 246", "161, 217", "56, 217",
    "86, 217", "106, 217", "116, 217", "126, 217", "136, 217", "140, 217",
    "192, 241", "43, 222", "43, 217", "34, 217", "249, 135", "242, 145",
    "51, 217", "151, 220", "76, 217", "81, 18", "186, 246", "186, 250",
    "192, 246", "180, 246", "186, 259"]

num_files = len(os.listdir(path_1))+1    # Count the number of files
test_location = test_location_ground


#print(test_location)
X_gt, y1_gt, y2_gt = GroundTruthData(path_1, floor, test_location,num_files)

path_2 = '/content/Localization_project-main/Ground/beacons_pd'
num_files = len(os.listdir(path_2))+1    # Count the number of files
X_pd_custom, y1_pd_custom, y2_pd_custom = Custom_PD_data(path_2,floor,test_location_pd,num_files)
X_pd, y1_pd, y2_pd = GPPredictedData(path_2,num_files)
#print(num_files)
#print(X_pd_custom)
# #### Split the data set for training and testing ###########
X_train, X_test, y1_train, y1_test = train_test_split(X_pd, y1_pd,test_size=0.2, random_state= 7)
print(X_train)
print(X_gt)
mae_pd_x, mae_pd_custom_x, mae_gt_x, y1_pred, y1_pred_custom, y1_pred_gt = knnXaxis(X_train, y1_train, X_test,y1_test, y1_gt)
#print(y1_test)
# print(X_pd)
# #### Split the data set for training and testing ###########
X_train, X_test, y2_train, y2_test = train_test_split(X_pd, y2_pd,test_size=0.2, random_state= 7)
mae_pd_y, mae_pd_custom_y, mae_gt_y, y2_pred, y2_pred_custom, y2_pred_gt =  knnYaxis(X_train, y2_train, X_test, y2_test,y2_gt)
#print(y1_test,y2_test)
# print(y1_gt.to_frame()['cordX'], y2_gt.to_frame()['cordY'])
#y1_y2_gt = pd.concat([y1_gt.to_frame(), y2_gt.to_frame()], axis=1)
# print(y1_y2_gt)
y1_y2_pd_gt = pd.concat([pd.DataFrame(y1_pred_gt), pd.DataFrame(y2_pred_gt)], axis=1)
y1_y2_pd_gt.rename(columns={'0': 'cordX', '0': 'cordY'})
# print(pd.DataFrame(y1_pred_gt))
# print(y1_y2_pd_gt)
# print(pd.concat([y1_y2_gt, y1_y2_pd_gt], axis=1))
print("---------KNN Localization performance evaluatiion ---------" )
path_3 = floor+'/results'
path_4 = floor+'/results'
mse_knn_pd = knnLocalizationPredictedData(y1_test, y2_test, X_test, y1_pred, y2_pred, path_3)
mse_knn_custom_pd = knnLocalizationcustomPD(y1_pd_custom, y2_pd_custom, X_pd_custom, y1_pred_custom, y2_pred_custom)
#mse_knn_gt = knnLocalizationGroundTruthData(y1_gt, y2_gt, X_gt, y1_pred_gt, y2_pred_gt, path_4)

/tmp/ipython-input-10-43222634.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-97.7' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataframe_gt.fillna('-97.7', inplace=True)


       IW25030290  IW25030183  IW25030321  IW25030294  IW25030952  IW25030316  \
17776      -102.0     -102.00      -92.39      -102.0     -102.00      -90.17   
18342      -102.0     -102.00     -102.00      -102.0     -102.00     -102.00   
34174      -102.0      -91.53     -102.00      -102.0      -89.50     -102.00   
34758      -102.0      -93.05     -102.00      -102.0      -94.00     -102.00   
1101       -102.0     -102.00     -102.00      -102.0     -102.00     -102.00   
...           ...         ...         ...         ...         ...         ...   
38467      -102.0     -102.00     -102.00      -102.0     -102.00     -102.00   
10742      -102.0     -102.00     -102.00      -102.0     -102.00      -92.19   
49689      -102.0      -89.37     -102.00      -102.0      -90.98     -102.00   
58564      -102.0     -102.00     -102.00      -102.0     -102.00     -102.00   
61615      -102.0     -102.00     -102.00      -102.0     -102.00     -102.00   

       IW25030298  IW250303

NameError: name 'y1_pred' is not defined

Localization_3 (for path estimation)

In [ ]:
from re import X
import os
from IPython.display import display
from itertools import product
import numpy as np
from sklearn.metrics import mean_absolute_error
import pandas as pd
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import accuracy_score
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C,WhiteKernel , RationalQuadratic ,ExpSineSquared
# %matplotlib notebook



def GPPredictedData(directory_pd, num_files):
    dataframes_pd = []
    for filename in os.listdir(directory_pd):
        file_path = os.path.join(directory_pd, filename)
        # Read CSV file
        df = pd.read_csv(file_path)
        # Rename column
        df.rename(columns={'rssi_mean': filename.split('.csv')[0]}, inplace=True)
        # Drop 'rssi_std' column
        df = df.drop(['rssi_std'], axis=1)
        # Append dataframe to the list
        dataframes_pd.append(df)
    # Merge all dataframes on 'location'
    dataframes_pd = pd.concat([df.set_index('location') for df in dataframes_pd], axis=1, join='inner').reset_index()
    new = dataframes_pd.location.str.split(",", n = -1, expand = True)
    dataframes_pd['cordX'] = new[0].astype(int)
    dataframes_pd['cordY'] = new[1].astype(int)
    X_pd = dataframes_pd.iloc[:, 1:num_files]  # Adjusted slice to include all device columns (assuming num_files is the count of device columns)
    y1_pd = dataframes_pd['cordX']
    y2_pd = dataframes_pd['cordY']

    # Sort the columns of X_pd in ascending order (alphabetical order of device names)
    sorted_columns = sorted(X_pd.columns)
    X_pd = X_pd[sorted_columns]

    dataframes_pd.head()
    dataframes_pd.shape

    return X_pd, y1_pd, y2_pd



def ContinuousData(num_files):
    dataframes_pd = []
    file_path = '/content/filled_Ground_A-block.csv'
        # Read CSV file
    dataframes_pd = pd.read_csv(file_path)
        # Rename column

    # Merge all dataframes on 'location'
    #dataframes_pd = pd.concat([df.set_index('location') for df in dataframes_pd], axis=1, join='inner').reset_index()
    new = dataframes_pd.location.str.split(",", n = -1, expand = True)
    dataframes_pd['cordX'] = new[0].astype(int)
    dataframes_pd['cordY'] = new[1].astype(int)
    X_cd = dataframes_pd.iloc[:, 1:num_files]
    y1_cd = dataframes_pd['cordX']
    y2_cd = dataframes_pd['cordY']

    dataframes_pd.head()
    dataframes_pd.shape

    return X_cd, y1_cd, y2_cd


path_2 = '/content/Localization_project-main/Ground/beacons_pd'
num_files = len(os.listdir(path_2))+1    # Count the number of files
X_cd, y1_cd, y2_cd = ContinuousData(num_files)
X_pd, y1_pd, y2_pd = GPPredictedData(path_2,num_files)

X_train, X_test, y1_train, y1_test = train_test_split(X_pd, y1_pd,test_size=0.2, random_state= 7)
# print(X_train)
# print(X_cd)
knn_classifier = KNeighborsClassifier(n_neighbors = 3)
knn_classifier.fit(X_train, y1_train.values.ravel())
y1_pred=knn_classifier.predict(X_cd)
X_train, X_test, y2_train, y2_test = train_test_split(X_pd, y2_pd,test_size=0.2, random_state= 7)
knn_classifier.fit(X_train, y2_train.values.ravel())
y2_pred=knn_classifier.predict(X_cd)
print(y1_pred)
print(y2_pred)

[37 49 31 37 90 77 77 90 77 77 54 77 89 89 89 59 77 77 89 77 40 89 83 77
 40 77 89 83 58 30]
[82 73 82 82 79 52 52 79 52 52 71 52 73 73 73 69 52 52 73 52 79 73 72 52
 79 52 73 72 70 79]


For plotting path

In [ ]:
import matplotlib.pyplot as plt

# Given coordinates
x = [37, 49, 31, 37, 90, 77, 77, 90, 77, 77, 54, 77, 89, 89, 89, 59, 77, 77, 89, 77, 40, 89, 83, 77, 40, 77, 89, 83, 58, 30]
y = [82, 73, 82, 82, 79, 52, 52, 79, 52, 52, 71, 52, 73, 73, 73, 69, 52, 52, 73, 52, 79, 73, 72, 52, 79, 52, 73, 72, 70, 79]

# Create plot
plt.figure(figsize=(12, 8))
plt.scatter(x, y, c='blue', s=100, alpha=0.7, edgecolors='w', linewidth=2)

# Add labels and title
plt.title('Coordinate Points Plot', fontsize=16)
plt.xlabel('X Coordinates', fontsize=14)
plt.ylabel('Y Coordinates', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)

# Add point labels
for i, (xi, yi) in enumerate(zip(x, y)):
    plt.annotate(f'P{i+1}', (xi, yi), xytext=(5, 5),
                 textcoords='offset points', fontsize=9)

plt.tight_layout()
plt.show()